In [4]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.externals import joblib

## Chronic_Kidney_Disease Data Set 

### Data

In [5]:
file = r'./data/chronic_kidney_disease.csv';

In [6]:
df = pd.read_csv(file, header=0, skip_blank_lines=True, skipinitialspace=True)

### Data understanding

In [7]:
df.head()

,age,blood_pressure,specific_gravity,albumin,sugar,red_blood_cell,pus_cell,pus_cell_clumps,backeria,blood_glocuse_random,...,packed_cell_volume,White_BC_count,Red_BC_count,hypertension,diabetes_mellitus,coronary_artery_disease,appetite,pedal_edema,anemia,class
0,48,80,1.02,1,0,?,normal,notpresent,notpresent,121,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7,50,1.02,4,0,?,normal,notpresent,notpresent,?,...,38,6000,?,no,no,no,good,no,no,ckd
2,62,80,1.01,2,3,normal,normal,notpresent,notpresent,423,...,31,7500,?,no,yes,no,poor,no,yes,ckd
3,48,70,1.005,4,0,normal,abnormal,present,notpresent,117,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51,80,1.01,2,0,normal,normal,notpresent,notpresent,106,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [8]:
df = df.replace('?', np.NaN)

In [9]:
df.head()

,age,blood_pressure,specific_gravity,albumin,sugar,red_blood_cell,pus_cell,pus_cell_clumps,backeria,blood_glocuse_random,...,packed_cell_volume,White_BC_count,Red_BC_count,hypertension,diabetes_mellitus,coronary_artery_disease,appetite,pedal_edema,anemia,class
0,48,80,1.02,1,0,NaN,normal,notpresent,notpresent,121,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7,50,1.02,4,0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,62,80,1.01,2,3,normal,normal,notpresent,notpresent,423,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,48,70,1.005,4,0,normal,abnormal,present,notpresent,117,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51,80,1.01,2,0,normal,normal,notpresent,notpresent,106,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [10]:
df.shape

(400, 25)

In [11]:
df.describe()

,age,blood_pressure,specific_gravity,albumin,sugar,red_blood_cell,pus_cell,pus_cell_clumps,backeria,blood_glocuse_random,...,packed_cell_volume,White_BC_count,Red_BC_count,hypertension,diabetes_mellitus,coronary_artery_disease,appetite,pedal_edema,anemia,class
count,391,388,353,354,351,248,335,396,396,356,...,329,294,269,398,398,398,399,399,399,400
unique,76,10,5,6,6,2,2,2,2,146,...,42,89,45,2,2,2,2,2,2,2
top,60,80,1.02,0,0,normal,normal,notpresent,notpresent,99,...,41,9800,5.2,no,no,no,good,no,no,ckd
freq,19,116,106,199,290,201,259,354,374,10,...,21,11,18,251,261,364,317,323,339,250


In [12]:
# count the number of NaN values in each column
df.isnull().sum()

age                          9
blood_pressure              12
specific_gravity            47
albumin                     46
sugar                       49
red_blood_cell             152
pus_cell                    65
pus_cell_clumps              4
backeria                     4
blood_glocuse_random        44
blood_urea                  19
serum_createnine            17
sodium                      87
potassium                   88
hemoglobin                  52
packed_cell_volume          71
White_BC_count             106
Red_BC_count               131
hypertension                 2
diabetes_mellitus            2
coronary_artery_disease      2
appetite                     1
pedal_edema                  1
anemia                       1
class                        0
dtype: int64

### Data preparation

In [13]:
# to a numeric type. if columns cannot be converted - left alone
df = df.apply(pd.to_numeric, errors='ignore')

In [14]:
df.dtypes

age                        float64
blood_pressure             float64
specific_gravity           float64
albumin                    float64
sugar                      float64
red_blood_cell              object
pus_cell                    object
pus_cell_clumps             object
backeria                    object
blood_glocuse_random       float64
blood_urea                 float64
serum_createnine           float64
sodium                     float64
potassium                  float64
hemoglobin                 float64
packed_cell_volume         float64
White_BC_count             float64
Red_BC_count               float64
hypertension                object
diabetes_mellitus           object
coronary_artery_disease     object
appetite                    object
pedal_edema                 object
anemia                      object
class                       object
dtype: object

In [15]:
#INTEGER
# fill NaN with MEAN (for Integer)
intFeatures = ['age','blood_pressure','blood_glocuse_random', 'blood_urea', 'sodium', 'packed_cell_volume','White_BC_count']
df[intFeatures] = df[intFeatures].apply(lambda x: x.fillna(int(x.mean())),axis=0)

In [16]:
# convert to integer type
df[intFeatures] = df[intFeatures].astype(int)

In [17]:
#FLOAT
# fill NaN with MEAN (for float)
floatFeatures = ['serum_createnine','potassium','hemoglobin','Red_BC_count']
df[floatFeatures] = df[floatFeatures].apply(lambda x: x.fillna(x.mean()),axis=0)

In [18]:
# round float
df = df.round({'serum_createnine': 1, 'potassium': 1, 'hemoglobin': 1, 'Red_BC_count': 1})

In [19]:
# convert 'class': string to integer //todo LE or OHE
classMap = {'ckd': 1, 'notckd': 2}
df['class'] = df['class'].map(classMap)

In [20]:
# CATEGORICAL
# for categorical:
catFeatures = ['specific_gravity', 'albumin', 'sugar', 'red_blood_cell', 'pus_cell', 'pus_cell_clumps', 'backeria', 'appetite']
# fill NaN with "missing" - add new category
df[catFeatures] = df[catFeatures].apply(lambda x: x.fillna('missing'),axis=0)

In [21]:
# Convert categorical variable into dummy/indicator variables
df_cat = pd.get_dummies(df[catFeatures])

In [22]:
# concat dummy/indicator variables
df = pd.concat([df, df_cat], axis=1)

In [23]:
# drop useless categorical column
df=df.drop(catFeatures, axis=1)

In [24]:
# BOOLEAN
booleanFeatures = ['hypertension', 'diabetes_mellitus', 'coronary_artery_disease', 'pedal_edema', 'anemia']
# fill NaN with "missing" - add new category
df[booleanFeatures] = df[booleanFeatures].apply(lambda x: x.fillna('missing'),axis=0)

In [25]:
# Convert boolean(actualy categorical - with 'missing') variable into dummy/indicator variables
df_bool = pd.get_dummies(df[booleanFeatures])

In [26]:
# concat dummy/indicator variables
df = pd.concat([df, df_bool], axis=1)

In [27]:
# drop useless categorical column
df=df.drop(booleanFeatures, axis=1)

In [28]:
# integer features scaling
scaler = MinMaxScaler()
df[intFeatures] = scaler.fit_transform(df[intFeatures])

In [29]:
# float features scaling
df[floatFeatures] = scaler.fit_transform(df[floatFeatures])

In [30]:
# split features and label(class)
dfX = df.drop(['class'], axis=1)
dfY = df['class']

### final data

In [31]:
# get features names
fNames = dfX.columns

In [32]:
# Get values
# features
F = dfX[fNames].values
# targets
t = dfY.values

In [33]:
#filename = "LA_Chronic_Kidney_Disease_DataPreparation_v0.1.0"

In [34]:
#joblib.dump(X, file_name + '_features.gz')
#joblib.dump(y, file_name + '_target.gz')

In [35]:
#F = joblib.load('LA_Chronic_Kidney_Disease_DataPreparation_v0.1.0_features.gz')
#t = joblib.load('LA_Chronic_Kidney_Disease_DataPreparation_v0.1.0_target.gz')

In [36]:
F.shape, t.shape

((400L, 61L), (400L,))

In [37]:
t = t.reshape(-1, 1)

In [38]:
F.shape, t.shape

((400L, 61L), (400L, 1L))

### train_test_split

In [39]:
X, X_test, y, y_test = train_test_split(F, t, train_size=0.7, test_size=0.3, random_state=1)

In [40]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape

(280L, 2L)

### simple neural network building

In [41]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [42]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [43]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    return J

The forward-propagate function computes the hypothesis for each training instance given the current parameters.  It's output shape should match the same of our one-hot encoding for y.

In [44]:
# initial setup
input_size = 61
hidden_size = 25
num_labels = 2
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

((25L, 62L), (2L, 26L))

In [45]:
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((280L, 62L), (280L, 25L), (280L, 26L), (280L, 2L), (280L, 2L))

The cost function, after computing the hypothesis matrix h, applies the cost equation to compute the total error between y and h.

In [46]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)

1.4319808214843752

Our next step is to add regularization to the cost function.

In [47]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [48]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)

1.4472140780633294

Next up is the backpropagation algorithm.  Backpropagation computes the parameter updates that will reduce the error of the network on the training data.  The first thing we need is a function that computes the gradient of the sigmoid function we created earlier.

In [49]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

Now we're ready to implement backpropagation to compute the gradients.  Since the computations required for backpropagation are a superset of those required in the cost function, we're actually going to extend the cost function to also perform backpropagation and return both the cost and the gradients.

In [50]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 62)
    delta2 = np.zeros(theta2.shape)  # (2, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 62)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 2)
        yt = y[t,:]  # (1, 2)
        
        d3t = ht - yt  # (1, 2)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

The hardest part of the backprop computation is getting the matrix dimensions right.

In [51]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)
J, grad.shape

(1.4472140780633294, (1602L,))

We still have one more modification to make to the backprop function - adding regularization to the gradient calculations.  The final regularized version is below.

In [52]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 62)
    delta2 = np.zeros(theta2.shape)  # (2, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 62)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 2)
        yt = y[t,:]  # (1, 2)
        
        d3t = ht - yt  # (1, 2)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [53]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)
J, grad.shape

(1.4472140780633294, (1602L,))

We're finally ready to train our network and use it to make predictions.

In [54]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 400})
fmin

     fun: 0.17431567245544699
     jac: array([ -1.36437861e-08,  -3.64209874e-09,   8.54480201e-09, ...,
         3.86422331e-08,   5.30242338e-07,  -1.39246317e-07])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 400
     nit: 38
  status: 1
 success: True
       x: array([-1.47457246,  0.03369324, -0.05664917, ...,  0.84405336,
        0.84396547, -0.84443199])

We put a bound on the number of iterations since the objective function is not likely to completely converge. Let's use the parameters it found and forward-propagate them through the network to get some predictions.

In [55]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)

In [56]:
f1_score(y, y_pred, average='macro')  

1.0

### TEST

In [57]:
X_test = np.matrix(X_test)

In [58]:
X_test.shape

(120L, 61L)

In [59]:
a1, z2, a2, z3, h = forward_propagate(X_test, theta1, theta2)
y_test_pred = np.array(np.argmax(h, axis=1) + 1)

In [60]:
y_test_pred.shape

(120L, 1L)

In [61]:
f1_score(y_test, y_test_pred, average='macro')  

0.99145238264833679

In [62]:
f1_score(y_test, y_test_pred, average='micro')  

0.9916666666666667

In [63]:
f1_score(y_test, y_test_pred, average='weighted') 

0.9916779447728945

In [64]:
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          1       1.00      0.99      0.99        70
          2       0.98      1.00      0.99        50

avg / total       0.99      0.99      0.99       120



And we're done!  We've successfully implemented a rudimentary feed-forward neural network with backpropagation and used it to classify Chronic Kidney Disease.